# Imputation of missing values

This is a component for imputing missing values using mean, median or most frequent. It makes use of an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the SDK to load a model and other artifacts.
- how to use a model to provide real-time transformations.

In [ ]:
%%writefile Transformer.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Transformer(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        logger.info("Initializing")

        # Load Artifacts: Estimator, etc
        model = load_model(experiment_id=experiment_id)
        self.estimator = model["estimator"]
        self.columns = model["columns"]
        self.strategy = model["strategy"]
        self.numerical_indexes = model["numerical_indexes"]

        logger.info("Init complete!")

    def class_names(self):
        return self.columns[~pd.isna(self.estimator.statistics_)].tolist()

    def transform_input(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Returns a transformation on input data.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.

        Returns:
            numpy.array: Array-like with transformations.
        """
        # Put data in a pandas.DataFrame
        df = pd.DataFrame(X, columns=feature_names)

        # Replace None with np.nan
        df.fillna(value=pd.np.nan, inplace=True)

        # Put data back in a numpy.ndarray
        X = df[self.columns].to_numpy()

        # Perform Transformation
        if self.strategy == "mean" or self.strategy == "median":
            X[:, self.numerical_indexes] = self.estimator.transform(X[:, self.numerical_indexes])
        elif self.strategy == "most_frequent":
            X = self.estimator.transform(X)
        return X

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%writefile env
export DATASET="${DATASET:-titanic}"
export TARGET="${TARGET:-Survived}"
export EXPERIMENT_ID="${EXPERIMENT_ID:-b48b03b1-5581-4e1d-a429-f31a34f78e1c}"
export MODEL_NAME="Transformer"
export API_TYPE="REST"
export SERVICE_TYPE="TRANSFORMER"
export PERSISTENCE=0
export LOG_LEVEL="DEBUG"
read -r -d "" PARAMETERS << EOM
[{"type": "STRING", "name": "dataset", "value": "$DATASET"},
 {"type": "STRING", "name": "target", "value": "$TARGET"},
 {"type": "STRING", "name": "experiment_id", "value": "$EXPERIMENT_ID"}]
EOM
export PARAMETERS

In [ ]:
%%bash
. ./env
seldon-core-microservice "$MODEL_NAME" "$API_TYPE" \
    --service-type "$SERVICE_TYPE" \
    --persistence "$PERSISTENCE" \
    --parameters "$PARAMETERS" \
    --log-level "$LOG_LEVEL" > log.txt 2>&1 &

ATTEMPT=0
until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    # exit process if not healthy after 10 seconds
    if [ "$ATTEMPT" -gt 10 ]; then
        cat log.txt
        exit 1
    fi
    ATTEMPT=$((ATTEMPT + 1))
    sleep 1
done
echo "Deployment successful. Waiting for requests."

## Make transformations

In [ ]:
%%bash
curl -sSL localhost:5000/transform-input --data-binary @- << EOF
json={
    "data": {
        "names": ["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"],
        "ndarray": [
            [123, 2, "Nasser, Mr. Nicholas", "male", null, 1, 0, "237736", 30.0708, null, "C"]
        ]
    }
}
EOF

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill